# Fitting impedance spectra

## 1. Import and initialize equivalent circuit(s)

To begin we will import the Randles' circuit and a custom circuit from the impedance package. A full list of currently available circuits are available in the [documentation](https://impedancepy.readthedocs.io/en/latest/circuits.html).

In [ ]:
import sys

sys.path.append("../../../")

from impedance.models.circuits import CustomCircuit, Randles

The classes we just imported represent different equivalent circuit models. To actually use them we want to initialize a specific instance and provide an initial guess for the parameters and any other options.

*E.g. for the randles circuit, one of the options is for a constant phase element (CPE) instead of an ideal capacitor.*

In [ ]:
randles = Randles(initial_guess=[0.01, 0.005, 0.1, 0.001, 200])
randlesCPE = Randles(initial_guess=[0.01, 0.005, 0.1, 0.9, 0.001, 200], CPE=True)

Defining the custom circuit works a little differently. Here we pass a string comprised of the circuit elements grouped either in series (separated with a `-`) or in parallel (using the form `p(X,Y)`). Each element can be appended with an integer (e.g. `R0`) or an underscore and an integer (e.g. `CPE_1`) to make keeping track of multiple elements of the same type easier.

In [ ]:
customCircuit = CustomCircuit(
    initial_guess=[0.01, 0.005, 0.1, 0.005, 0.1, 0.001, 200],
    circuit="R_0-p(R_1,C_1)-p(R_2,C_2)-Wo_1",
)

As of version 0.4, you can now specify values you want to hold constant. For example,

In [ ]:
customConstantCircuit = CustomCircuit(
    initial_guess=[None, 0.005, 0.1, 0.005, 0.1, 0.001, None],
    constants={"R_0": 0.02, "Wo_1_1": 200},
    circuit="R_0-p(R_1,C_1)-p(R_2,C_2)-Wo_1",
)

Each of the circuit objects we create can be printed in order to see the properties that have been defined for that circuit.

In [ ]:
print(customConstantCircuit)

## 2. Formulate data

Several convenience functions for importing data exist in the impedance.preprocessing module, including one for reading simple `.csv` files where frequencies are stored in the first column, real parts of the impedance are in the second column, and imaginary parts of the impedance are in the third column.

In [ ]:
from impedance import preprocessing

frequencies, Z = preprocessing.readCSV("../../../data/exampleData.csv")

# keep only the impedance data in the first quandrant
frequencies, Z = preprocessing.ignoreBelowX(frequencies, Z)

## 3. Fit the equivalent circuits to a spectrum

Each of the circuit classes has a `.fit()` method which finds the best fitting parameters.

After fitting a circuit, the fit parameters rather that the inital guesses are shown when printing.

In [ ]:
randles.fit(frequencies, Z)
randlesCPE.fit(frequencies, Z)
customCircuit.fit(frequencies, Z)
customConstantCircuit.fit(frequencies, Z)

print(customConstantCircuit)

## 4a. Predict circuit model and visualize with matplotlib

In [ ]:
import matplotlib.pyplot as plt
from impedance.visualization import plot_nyquist

f_pred = np.logspace(5, -2)

randles_fit = randles.predict(f_pred)
randlesCPE_fit = randlesCPE.predict(f_pred)
customCircuit_fit = customCircuit.predict(f_pred)
customConstantCircuit_fit = customConstantCircuit.predict(f_pred)

fig, ax = plt.subplots(figsize=(5, 5))

plot_nyquist(ax, Z)
plot_nyquist(ax, randles_fit, fmt="-")
plot_nyquist(ax, randlesCPE_fit, fmt="-")
plot_nyquist(ax, customCircuit_fit, fmt="-")
plot_nyquist(ax, customConstantCircuit_fit, fmt="-")

ax.legend(
    [
        "Data",
        "Randles",
        "Randles w/ CPE",
        "Custom Circuit",
        "Custom Circuit w/ Constant R0 and W1_1",
    ]
)
plt.show()

## 4b. Or use the convenient plotting method included in the package

In [ ]:
randles.plot(f_data=frequencies, Z_data=Z, kind="nyquist")
randlesCPE.plot(f_data=frequencies, Z_data=Z, kind="nyquist")
customCircuit.plot(f_data=frequencies, Z_data=Z, kind="nyquist")
customConstantCircuit.plot(f_data=frequencies, Z_data=Z, kind="nyquist")

plt.show()